In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from keras import Sequential, callbacks, layers
from keras.src.optimizers import Adam

from beast.infra import Constants
from beast.neural_networks import NetFactory, NetVersion
from notebooks.infra import PgnConverter, save_onnx_with_metadata, set_seed

In [ ]:
model_version = NetVersion.V1
net = NetFactory.from_version(model_version)
x_data, y_data = PgnConverter.load_training_data_from_file(
    Constants.DATA_PATH / "evaluated" / "evaluated_games.txt", get_input=net.fen_to_input
)

In [ ]:
set_seed(42)

model = Sequential([
    layers.Input(x_data[0].shape),
    layers.Conv2D(128, kernel_size=3),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
    layers.Conv2D(256, kernel_size=3),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
    layers.Flatten(),
    layers.Dense(256),
    layers.LeakyReLU(),
    layers.Dense(1, activation="tanh"),
])

adam = Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss="mse", metrics=["acc", "mape", "mae"])
early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=200)

history = model.fit(
    x_data,
    y_data,
    epochs=2000,
    batch_size=256,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1,
)

plt.figure()
plt.plot(history.history["loss"], label="Train")
plt.plot(history.history["val_loss"], label="Val")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.grid(visible=True)
plt.show()

In [ ]:
save_onnx_with_metadata(model, "test_model", x_data[0], Path(), model_version=model_version.name)